# Frontend for Anomaly Detection using Gradio 


In [ ]:
!pip install gradio

In [1]:
import gradio

c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import gradio as gr 
import tensorflow as tf
import numpy as np
import requests

#Load Model here like .h5 
#load labels.csv here 


In [6]:
import gradio as gr
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import os
import matplotlib.pyplot as plt
import shutil

# Load the model
# model = load_model('CNN_LSTM_10.h5')
model = load_model('cnn_lstm_model_optimized_reduced.h5')

def normalize_scores(scores):
    min_score = min(scores)
    max_score = max(scores)
    return [(score - min_score) / (max_score - min_score) for score in scores]

def process_video(video_path):
    # Create temporary directories
    temp_dir = 'temp_frames'
    anomaly_dir = 'anomaly_frames'
    os.makedirs(temp_dir, exist_ok=True)
    os.makedirs(anomaly_dir, exist_ok=True)

    # Create a copy of the uploaded video
    video_copy_path = 'uploaded_video.mp4'
    shutil.copy(video_path, video_copy_path)

    # Open the video file
    cap = cv2.VideoCapture(video_copy_path)
    frame_count = 0
    anomaly_scores = []
    timestamps = []
    anomaly_frames = []

    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define anomaly threshold (adjust as needed)
    anomaly_threshold = 0.90

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Calculate timestamp
        timestamp = frame_count / fps
        timestamps.append(timestamp)

        # Preprocess the frame
        processed_frame = cv2.resize(frame, (112, 112))
        processed_frame = processed_frame / 255.0

        # Make prediction
        prediction = model.predict(np.expand_dims(processed_frame, axis=0))
        anomaly_score = prediction[0][0]
        anomaly_scores.append(anomaly_score)

        # Save frame if it exceeds the anomaly threshold
        if anomaly_score > anomaly_threshold:
            frame_path = os.path.join(anomaly_dir, f'anomaly_frame_{frame_count:04d}_{timestamp:.2f}s.jpg')
            cv2.imwrite(frame_path, frame)
            # Append frame_path and a caption containing timestamp and anomaly score
            anomaly_frames.append((frame_path, f'Time: {timestamp:.2f}s, Score: {anomaly_score:.2f}'))

        frame_count += 1

    cap.release()

    # Normalize the anomaly scores
    normalized_scores = normalize_scores(anomaly_scores)

    # Create a plot of normalized anomaly scores with timestamps
    plt.figure(figsize=(12, 6))
    plt.plot(timestamps, normalized_scores)
    plt.title('Normalized Anomaly Scores Over Time')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Normalized Anomaly Score')
    plt.ylim(0, 1)

    # Add a secondary x-axis for frame numbers
    ax2 = plt.gca().twiny()
    ax2.set_xlabel('Frame')
    ax2.set_xlim(plt.gca().get_xlim())
    ax2.set_xticks(np.linspace(plt.gca().get_xlim()[0], plt.gca().get_xlim()[1], 5))
    ax2.set_xticklabels([f'{int(x)}' for x in np.linspace(0, frame_count, 5)])

    # Mark anomaly points on the plot
    for frame_path, caption in anomaly_frames:
        plt.plot(timestamps, normalized_scores, 'ro')

    plt.tight_layout()
    plot_path = 'anomaly_scores.png'
    plt.savefig(plot_path)
    plt.close()

    return video_copy_path, plot_path, anomaly_frames

# Gradio interface
iface = gr.Interface(
    fn=process_video,
    inputs=gr.Video(),
    outputs=[
        gr.Video(label="Uploaded Video"),
        gr.Image(type="filepath", label="Normalized Anomaly Scores"),
        gr.Gallery(label="Anomaly Frames")
    ],
    title="Video Anomaly Detection",
    description="Upload a video to detect anomalies. The video will be displayed alongside the normalized anomaly score plot and frames where anomalies were detected."
)

# iface.launch()
iface.launch(share=True)


Running on local URL:  http://127.0.0.1:7864


c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://084432e7649618373d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
